In [5]:
# 1. IMPORTS
import pandas as pd
import os
import glob
from datetime import datetime
import re
from ics import Calendar, Event
import pytz
import hashlib
from tabulate import tabulate # <-- Re-added for console display

# 2. CONFIGURATION
SCHEDULE_FILE_NAME = 'schedule.xlsx'
TIMEZONE = 'Asia/Kolkata'

COURSE_DETAILS_MAP = {
    'AN(A)': {'Faculty': 'Nitin Pillai', 'Venue': 'T6'}, 'AN(B)': {'Faculty': 'Nitin Pillai', 'Venue': 'T6'},
    'B2B(A)': {'Faculty': 'Sandip Trada', 'Venue': 'T5'}, 'B2B(B)': {'Faculty': 'Rupam Deb', 'Venue': '208-B'},
    "B2B('C)": {'Faculty': 'Nityesh Bhatt', 'Venue': '208-B'}, 'BS': {'Faculty': 'Satish Nair', 'Venue': 'T6'},
    'CC&AU(A)': {'Faculty': 'Lalit Arora', 'Venue': 'T6'}, 'CC&AU(B)': {'Faculty': 'Lalit Arora', 'Venue': 'T6'},
    'CSE': {'Faculty': 'Shahir Bhatt', 'Venue': 'T6'}, 'DADM': {'Faculty': 'Mahesh K C', 'Venue': 'T3'},
    'DC': {'Faculty': 'Sapan Oza', 'Venue': 'T6'}, 'DM(A)': {'Faculty': 'Shailesh Prabhu', 'Venue': 'T7'},
    'DM(B)': {'Faculty': 'Shailesh Prabhu', 'Venue': 'T7'}, "DRM('C)": {'Faculty': 'Pankaj Agrawal', 'Venue': 'T5'},
    'DRM(A)': {'Faculty': 'Bhavesh Patel', 'Venue': 'T6'}, 'DRM(B)': {'Faculty': 'Bhavesh Patel', 'Venue': 'T6'},
    "DV&VS('C)": {'Faculty': 'Anand Kumar', 'Venue': 'T5'}, 'DV&VS(A)': {'Faculty': 'Somayya Madakam', 'Venue': 'E3'},
    'DV&VS(B)': {'Faculty': 'Somayya Madakam', 'Venue': 'E3'}, 'DV&VS(D)': {'Faculty': 'Anand Kumar', 'Venue': 'T5'},
    'IMC(A)': {'Faculty': 'Sanjay Jain', 'Venue': 'T7'}, 'IMC(B)': {'Faculty': 'Riddhi Ambavale', 'Venue': 'T7'},
    'INB(A)': {'Faculty': 'M C Gupta', 'Venue': 'T6'}, 'INB(B)': {'Faculty': 'M C Gupta', 'Venue': 'T6'},
    'INB(C)': {'Faculty': 'M C Gupta', 'Venue': 'T6'}, 'LSS(A)': {'Faculty': 'Rajesh Jain', 'Venue': 'T3'},
    'LSS(B)': {'Faculty': 'Rajesh Jain', 'Venue': 'T3'}, 'ML&AI(A)': {'Faculty': 'Omkar Sahoo', 'Venue': 'T5'},
    'ML&AI(B)': {'Faculty': 'Omkar Sahoo', 'Venue': 'T5'}, 'OMSD': {'Faculty': 'Dinesh Panchal', 'Venue': 'T3'},
    'PDBE(A)': {'Faculty': 'Nina Muncherji', 'Venue': 'T7'}, 'PDBE(B)': {'Faculty': 'Nina Muncherji', 'Venue': 'T7'},
    "SCM('C)": {'Faculty': 'Praneti Shah', 'Venue': 'T3'}, 'SCM(A)': {'Faculty': 'Praneti Shah', 'Venue': 'T3'},
    'SCM(B)': {'Faculty': 'Praneti Shah', 'Venue': 'T3'}, 'SMKT(A)': {'Faculty': 'Himanshu Chauhan', 'Venue': 'T6'},
    'SMKT(B)': {'Faculty': 'Kavita Saxena', 'Venue': 'T5'}, 'TEOM(A)': {'Faculty': 'P Ganesh', 'Venue': 'T3'},
    'TEOM(B)': {'Faculty': 'P Ganesh', 'Venue': 'T3'}, "VALU('C)": {'Faculty': 'Dimple Bhojwani', 'Venue': 'T6'},
    'VALU(A)': {'Faculty': 'Dipti Saraf', 'Venue': 'T5'}, 'VALU(B)': {'Faculty': 'Dipti Saraf', 'Venue': 'T5'},
    'VALU(D)': {'Faculty': 'Dimple Bhojwani', 'Venue': 'T6'}
}

# 3. FUNCTIONS
def normalize_string(text):
    if isinstance(text, str):
        return text.replace(" ", "").replace("(", "").replace(")", "").replace("'", "").upper()
    return ""

def load_and_clean_schedule(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name=1, header=None, skiprows=3)
        schedule_df = df.iloc[:, 0:14].copy()
        schedule_df[0] = pd.to_datetime(schedule_df[0], errors='coerce')
        schedule_df.dropna(subset=[0], inplace=True)
        return schedule_df
    except Exception as e:
        print(f"ERROR: Could not load schedule file '{file_path}'. Details: {e}")
        return pd.DataFrame()

def get_student_info_from_files(roll_number_str, folder_path='.'):
    student_name, student_sections = None, []
    subject_files = [f for f in glob.glob(os.path.join(folder_path, '*.xlsx')) if os.path.basename(f) != SCHEDULE_FILE_NAME]
    print(f"Scanning {len(subject_files)} subject allocation file(s) for Roll No: {roll_number_str}...")
    search_roll_number = roll_number_str.upper()

    for file in subject_files:
        try:
            df = pd.read_excel(file, header=None)
            header_row_index = -1
            for i in range(min(5, len(df))):
                if df.iloc[i].astype(str).str.upper().str.contains('ROLL').any():
                    header_row_index = i; break
            if header_row_index == -1: continue
            
            subject_row = df.iloc[0]
            roll_no_columns = df.iloc[header_row_index][df.iloc[header_row_index].astype(str).str.upper().str.contains('ROLL')].index
            
            for col_idx in roll_no_columns:
                roll_number_column_as_text = df[col_idx].iloc[header_row_index + 1:].astype(str).str.upper()
                if search_roll_number in roll_number_column_as_text.values:
                    student_row_df = df[df[col_idx].astype(str).str.upper() == search_roll_number]
                    if not student_row_df.empty:
                        student_row_index = student_row_df.index[0]
                        section_name = subject_row[col_idx]
                        if student_name is None:
                            name_column_index = col_idx + 1
                            student_name = df.iloc[student_row_index, name_column_index]
                        student_sections.append(section_name)
                        print(f"  > Found in section: {section_name}")
        except Exception as e:
            print(f"  > ERROR: Could not process file '{os.path.basename(file)}'. Details: {e}")
            
    return student_name, list(set(student_sections))

def generate_ics_file(found_classes, filename):
    """Generates a robust iCalendar (.ics) file using UTC times for compatibility."""
    print(f"\nCreating calendar file with {len(found_classes)} events...")
    c = Calendar(creator="-//Student Timetable Script v3//EN")
    local_tz = pytz.timezone(TIMEZONE)

    for class_info in found_classes:
        try:
            e = Event()
            
            time_str = class_info['Time']
            am_pm = "AM" if "AM" in time_str else "PM"
            time_parts = time_str[:-2].split('-')
            start_time_str, end_time_str = f"{time_parts[0]}{am_pm}", f"{time_parts[1]}{am_pm}"

            start_dt_local = local_tz.localize(pd.to_datetime(f"{class_info['Date'].strftime('%Y-%m-%d')} {start_time_str}"))
            end_dt_local = local_tz.localize(pd.to_datetime(f"{class_info['Date'].strftime('%Y-%m-%d')} {end_time_str}"))
            
            if end_dt_local < start_dt_local:
                end_dt_local += pd.Timedelta(hours=12)

            e.begin = start_dt_local.astimezone(pytz.utc)
            e.end = end_dt_local.astimezone(pytz.utc)
            
            e.name = f"{class_info['Subject']}"
            e.location = class_info['Venue']
            e.description = f"Faculty: {class_info['Faculty']}"
            
            uid_string = f"{start_dt_local.isoformat()}-{class_info['Subject']}"
            e.uid = hashlib.md5(uid_string.encode('utf-8')).hexdigest() + "@timetable.script"
            
            c.events.add(e)
        except Exception as e:
            print(f"  > WARNING: Could not add event for {class_info.get('Subject', 'Unknown')}. Reason: {e}")
            
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(c.serialize())
    
    print(f"✅ Calendar file '{filename}' created successfully.")
    print("Find this file in your folder and import it into your calendar.")


# 4. MAIN SCRIPT EXECUTION
if __name__ == "__main__":
    master_schedule_df = load_and_clean_schedule(SCHEDULE_FILE_NAME)
    if not master_schedule_df.empty:
        user_roll_number = input("Enter the student's Roll Number: ").strip()
        student_name, student_sections = get_student_info_from_files(user_roll_number)
        
        if not student_sections:
            print(f"\nError: Roll Number '{user_roll_number}' not found.")
        else:
            print(f"\nFinding class times for {student_name} ({user_roll_number})...")
            
            NORMALIZED_COURSE_DETAILS_MAP = {normalize_string(section): details for section, details in COURSE_DETAILS_MAP.items()}
            normalized_student_section_map = {normalize_string(sec): sec for sec in student_sections}

            time_slots = {2: "8-9AM", 3: "9:10-10:10AM", 4: "10:20-11:20AM", 5: "11:30-12:30PM", 6: "12:30-1:30PM", 7: "1:30-2:30PM", 8: "2:40-3:40PM", 9: "3:50-4:50PM", 10: "5-6PM", 11: "6:10-7:10PM", 12: "7:20-8:20PM", 13: "8:30-9:30PM"}
            found_classes = []
            
            for index, row in master_schedule_df.iterrows():
                date, day = row[0], row[1]
                for col_index, time in time_slots.items():
                    cell_value = row[col_index]
                    if isinstance(cell_value, str):
                        normalized_cell = normalize_string(cell_value)
                        for norm_sec, orig_sec in normalized_student_section_map.items():
                            if norm_sec in normalized_cell:
                                details = NORMALIZED_COURSE_DETAILS_MAP.get(norm_sec, {'Faculty': 'N/A', 'Venue': '-'})
                                found_classes.append({"Date": date, "Day": day, "Time": time, "Subject": orig_sec, "Faculty": details['Faculty'], "Venue": details['Venue']})
            
            found_classes = [dict(t) for t in {tuple(d.items()) for d in found_classes}]

            if not found_classes:
                print("No classes found for this student in the master schedule.")
            else:
                # --- NEW: Display the console timetable for review ---
                print("\n--- Timetable Preview ---")
                timetable_df = pd.DataFrame(found_classes)
                timetable_df['Class Info'] = (timetable_df['Subject'] + '\n(' + 
                                              timetable_df['Faculty'].astype(str) + ')\n' + 
                                              'Venue: ' + timetable_df['Venue'].astype(str))
                timetable_df['Day/Date'] = timetable_df['Date'].dt.strftime('%a, %d-%b')
                sorted_times = [time_slots[key] for key in sorted(time_slots.keys())]
                final_schedule = timetable_df.pivot_table(index='Day/Date', columns='Time', values='Class Info', aggfunc='first')
                times_with_classes = [time for time in sorted_times if time in final_schedule.columns]
                final_schedule = final_schedule[times_with_classes]
                final_schedule.fillna('-', inplace=True)
                final_schedule = final_schedule.reindex(sorted(final_schedule.index, key=lambda x: pd.to_datetime(x.split(', ')[1] + " 2025")))
                
                print(tabulate(final_schedule, headers='keys', tablefmt='grid', stralign="center"))

                # --- NEW: Ask to create file with dynamic name ---
                add_choice = input(f"\nCreate an importable calendar file (.ics) for {student_name}? (y/n): ").lower()
                if add_choice == 'y':
                    sanitized_name = re.sub(r'[^a-zA-Z0-9_]', '', student_name.replace(" ", "_")).upper()
                    output_filename = f"{sanitized_name}_Timetable.ics"
                    generate_ics_file(found_classes, output_filename)

Enter the student's Roll Number:  24MBA407


Scanning 23 subject allocation file(s) for Roll No: 24MBA407...
  > Found in section: CSE
  > Found in section: DRM(A)
  > Found in section: DV&VS(A)
  > Found in section: INB(B)
  > Found in section: ML&AI(A)
  > Found in section: VALU(A)

Finding class times for Suriya Anvi Amit Vinita (24MBA407)...

--- Timetable Preview ---
+-------------+-----------------+----------------+-----------------+---------------+-------------------+-------------------+
|  Day/Date   |      8-9AM      |  9:10-10:10AM  |  10:20-11:20AM  |  1:30-2:30PM  |    3:50-4:50PM    |       5-6PM       |
+=============+=================+================+=================+===============+===================+===================+
| Mon, 29-Sep |     DRM(A)      |       -        |        -        |       -       |        CSE        |         -         |
|             | (Bhavesh Patel) |                |                 |               |  (Shahir Bhatt)   |                   |
|             |    Venue: T6    |            


Create an importable calendar file (.ics) for Suriya Anvi Amit Vinita? (y/n):  n
